In [33]:
import numpy as np
import pandas as pd

from copy import deepcopy
from collections import defaultdict

import camels
from camels import Game, state_from_dict

# Probability stats

In [2]:
def sampler(init_state):
    def sample_one():
        game = Game(init_state=deepcopy(init_state))
        game.one_round()
        return game.leaderboard()
    return sample_one

In [3]:
def stats(num, init_state):
    sample_one = sampler(init_state)
    camel_order = Game._leaderboard(init_state)
    res = {camel: np.zeros(len(camel_order), dtype=np.float64) for camel in camel_order}
    for _ in range(num):
        sample = sample_one()
        for i, camel in enumerate(sample):
            res[camel][i] += 1
    for v in res.values():
        v /= num
    return res

In [6]:
stats(10000, state_from_dict({1: ["a5", "a4", "a3", "a2", "a1"]}))

{'a5': array([0.4011, 0.2653, 0.1541, 0.0991, 0.0804]),
 'a4': array([0.2112, 0.257 , 0.1995, 0.163 , 0.1693]),
 'a3': array([0.1499, 0.1833, 0.2546, 0.2035, 0.2087]),
 'a2': array([0.1308, 0.1558, 0.2023, 0.2796, 0.2315]),
 'a1': array([0.107 , 0.1386, 0.1895, 0.2548, 0.3101])}

# Topology analysis

In [29]:
def extract_topologies(state):
    groups = group_positions(list(state))
    return set([hash_group(group, state) for group in groups])
        
def group_positions(positions):
    ordered = sorted(positions)
    groups = []
    group = []
    for i in ordered:
        if group and i - group[-1] > 2:
            groups.append(group)
            group = []
        group.append(i)
    return groups

def hash_group(group, state):
    res = []
    min_pos = group[0]
    max_pos = group[-1]
    for pos in range(min_pos, max_pos+1):
        stack = state.get(pos)
        size = len(stack.camels) if stack else 0
        res.append(size)
    return ",".join(map(str, res))

In [30]:
def sampler(camels):
    def sample_one():
        game = Game(camels=camels)
        topologies = defaultdict(int)
        while max(game.board._pos_to_stack) < 30:
            game.one_round()
            cur_tops = extract_topologies(game.board._pos_to_stack)
            for top in cur_tops:
                topologies[top] += 1
        return topologies
    return sample_one

In [41]:
def stats(num, camels):
    sample_one = sampler(camels)
    topologies = defaultdict(int)
    for _ in range(num):
        sample = sample_one()
        for top, count in sample.items():
            topologies[top] += count
    res = pd.DataFrame({"nums": list(topologies.values())}, index=list(topologies))
    return res.sort_values("nums", ascending=False)

In [44]:
stats(1000, ["a5", "a4", "a3", "a2", "a1"])

,nums
1,3770
2,909
"1,1",703
"1,0,1",423
3,211
"1,2",179
"2,1",143
"1,0,2",135
"1,1,1",109
"1,0,1,1",87
